In [1]:
import os
print(os.getcwd())  # this is where Jupyter is looking from
#one folder back
os.chdir('..')
print(os.getcwd())

c:\Users\Christian\Documents\Cand_merc\n1_masters_thesis\model_training
c:\Users\Christian\Documents\Cand_merc\n1_masters_thesis


In [2]:
import pandas as pd
import numpy as np
from load_data import load_dataset, ImageDataset

from split import get_dataloaders
from custom_data_aug import get_transforms

import torch
import torch.nn as nn
import torchvision
from config import CFG

from torchinfo import summary

from torch.utils.tensorboard import SummaryWriter
from training import train  # or wherever your updated function lives

from acc_loss_plot import plot_training_curves

In [3]:
# Load the dataframe
csv_path = "C:/Users/Christian/Desktop/N1_data/filtered_df.csv"
image_dir = "C:/Users/Christian/Desktop/N1_data/image_data/"
df, idx_to_label = load_dataset(csv_path, image_dir)

In [4]:
print(df.head())

                                           file_name label  label_idx
0  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
1  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
2  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
3  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
4  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0


In [5]:
train_transforms, val_transforms = get_transforms()

train_dataset, val_dataset, train_loader, val_loader = get_dataloaders(
    df, train_transforms, val_transforms
)

In [6]:
def build_model(device: torch.device, num_classes: int = CFG.NUM_CLASSES, fine_tune_from: int = -2) -> nn.Module:

    # Set the manual seeds
    torch.manual_seed(CFG.SEED)
    torch.cuda.manual_seed(CFG.SEED)
    """
    Builds a fine-tuned ResNet18 with a custom classifier head.

    Args:
        device: torch.device
        num_classes: number of output classes
        fine_tune_from: which ResNet layer (layer1–layer4) to unfreeze (e.g., -2 = unfreeze layer3 & layer4)
    """

    # Load pretrained ResNet18
    weights = torchvision.models.ResNet50_Weights.DEFAULT
    model = torchvision.models.resnet50(weights=weights).to(device)

    # Freeze all layers first
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze selected layers (e.g., layer3 and layer4)
    unfreeze_layers = [model.layer1, model.layer2, model.layer3, model.layer4][fine_tune_from:]
    for layer in unfreeze_layers:
        for param in layer.parameters():
            param.requires_grad = True

    # Replace the fully connected (fc) head
    model.fc = nn.Sequential(
        nn.Linear(2048, 512),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
    ).to(device)

    return model

In [7]:
cnn = build_model(device=CFG.DEVICE)

# View model summary
summary(
    model=cnn, 
    input_size=(CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [64, 3, 224, 224]    [64, 34]             --                   Partial
├─Conv2d (conv1)                         [64, 3, 224, 224]    [64, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [64, 64, 112, 112]   [64, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [64, 64, 112, 112]   [64, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [64, 64, 112, 112]   [64, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [64, 64, 56, 56]     [64, 256, 56, 56]    --                   False
│    └─Bottleneck (0)                    [64, 64, 56, 56]     [64, 256, 56, 56]    --                   False
│    │    └─Conv2d (conv1)               [64, 64, 56, 56]     [64, 64, 56, 56]     (4,096)              False
│    │    

In [8]:
# Define Loss Function
loss_fn = nn.CrossEntropyLoss(
    label_smoothing=0.1
)

# Define Optimizer
optimizer = torch.optim.Adam(
    cnn.parameters(),
    lr=CFG.LR
)

In [9]:
writer = SummaryWriter(log_dir="runs/exp1_efficientnet")

session = train(
    model=cnn,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=CFG.EPOCHS,
    device=CFG.DEVICE,
    writer=writer
)

writer.close()


Epoch 1/10


  0%|          | 0/117 [00:00<?, ?it/s]

: 

In [ ]:
# Convert history dict to DataFrame
session_history_df = pd.DataFrame(session_history)
session_history_df

In [ ]:
from acc_loss_plot import plot_training_curves
# Plot EfficientNet session training history 
plot_training_curves(session_history)